<a href="https://colab.research.google.com/github/jomericer/rhabdo_death_krt/blob/main/rhabdo_death_krt_datashare.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Init**

In [ ]:
! pip install --upgrade shap

import shap
import pandas as pd
import numpy as np
import os
import matplotlib.pylab as plt

from sklearn.ensemble import RandomForestClassifier
from matplotlib.pylab import rcParams
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve, auc, precision_score, recall_score, brier_score_loss, make_scorer, f1_score, fbeta_score, precision_recall_curve
from sklearn.model_selection import StratifiedKFold, train_test_split, cross_validate

%matplotlib inline

# load datasets
df_e = pd.read_csv('/df_e.csv')
df_m = pd.read_csv('/df_e.csv')
df_c = pd.read_csv('/df_e.csv')
print('datasets loaded')

# setup inputs and output
invars = ['age','aniongap','bicarbonate','urea','calcium','sodium','chloride','creatinine','potassium','phosphate','ck',
       'hematocrit','sex_F','ethnicity_white','wbc','platelet']
outvar = 'composite_rrt_death' # either composite_rrt_death or aki_7d_cr
print(f"total features: {len(invars)};  output = {outvar}\n{invars}")

# determine amount of class imbalance
db=['mimic','eicu','combined']
avg=[0]*3
for i, df_ in enumerate([df_m, df_e, df_c]):
  len_=df_.shape[0]
  pos=sum(df_[outvar])
  neg=len_-pos
  avg[i]=neg/pos
  print(f'{db[i]}: len={len_}, pos(1)={pos}, neg(0)={neg} => {pos/(len_)*100:.2f}%, scale_pos_weight={neg/pos:.2f}')
print(f'avg scale_pos_weight={np.mean(avg):.2f}')

# generate train and test datasets
split = 0.3
RAND = 15
folds = 3
m1_df = df_e # train/test dataset
val = df_m # validate dataset
X = m1_df.loc[:,invars].copy()
y = m1_df[outvar].copy().fillna(0)
X__test = val.loc[:,invars].copy()
y__test = val[outvar].copy()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=split, stratify=y, random_state=RAND)
print(f"\ntotal features: {len(invars)};  output = {outvar} -> total 1's = {y.sum()} ({y.sum()/len(y)*100:.1f}%)\ntrain size = {1-split}, test size = {split}")
print("Train and Test datasets created")


**ML model**

In [ ]:
rfc = RandomForestClassifier(
    n_estimators=60,
    criterion='entropy',
    max_depth=4,
    min_samples_split=5,
    class_weight="balanced_subsample",
    ccp_alpha=0.016634211826061396,
    max_samples=None,
    verbose=0)

print(f"Performing CV with {folds}-folds...")
cv_scores = cross_validate(rfc, X_train, y_train,
                           scoring=['accuracy','roc_auc','neg_log_loss'],
                           cv=StratifiedKFold(folds), return_train_score=True, verbose=0)
print(f"CV using Stratified Kfold strategy:\n \
Train accuracy: {np.mean(cv_scores['train_accuracy']):.4f}, ({np.std(cv_scores['train_accuracy']):.4f}) \n \
 Test accuracy: {np.mean(cv_scores['test_accuracy']):.4f}, ({np.std(cv_scores['test_accuracy']):.4f}) \n \
Train AUC: {np.mean(cv_scores['train_roc_auc']):.4f}, ({np.std(cv_scores['train_roc_auc']):.4f}) \n \
 Test AUC: {np.mean(cv_scores['test_roc_auc']):.4f}, ({np.std(cv_scores['test_roc_auc']):.4f}) \n \
Train logloss: {np.mean(cv_scores['train_neg_log_loss']):.4f}, ({np.std(cv_scores['train_neg_log_loss']):.4f}) \n \
 Test logloss: {np.mean(cv_scores['test_neg_log_loss']):.4f}, ({np.std(cv_scores['test_neg_log_loss']):.4f}) ")

# fit model
rfc.fit(X_train, y_train)
train_pred = rfc.predict(X_train)
train_prob = rfc.predict_proba(X_train)[:,1]
test_pred = rfc.predict(X_test)
test_prob = rfc.predict_proba(X_test)[:,1]
val_pred = rfc.predict(X__test)
val_prob = rfc.predict_proba(X__test)[:,1]

# Metrics
fpr_m, tpr_m, thr = roc_curve(comb_df_imp[outvar], comb_df_imp['mcmahon'])
optimal_idx = np.argmax(tpr_m - fpr_m)
AUCm  = auc(fpr_m, tpr_m)

print(f"\n{'FINAL TRAINED MODEL':26} {'train':10} | {'test':10} | {'validate':10} | mcmahon {thr[optimal_idx]}")
print(f" {'Accuracy:':25} {accuracy_score(y_train, train_pred):10.4f} | " \
      f"{accuracy_score(y_test, test_pred):10.4f} | " \
      f"{accuracy_score(y__test, val_pred):10.4f} |")
print(f" {'Precision Score:':25} {precision_score(y_train, train_pred):10.4f} | " \
      f"{precision_score(y_test, test_pred):10.4f} | " \
      f"{precision_score(y__test, val_pred):10.4f} | ")
print(f" {'Recall (sensitivity):':25} {recall_score(y_train, train_pred):10.4f} | " \
      f"{recall_score(y_test, test_pred):10.4f} | " \
      f"{recall_score(y__test, val_pred):10.4f} | {tpr_m[optimal_idx]:10.4f}")
print(f" {'AUC Score:':25} {roc_auc_score(y_train, train_prob):10.4f} | " \
      f"{roc_auc_score(y_test, test_prob):10.4f} | " \
      f"{roc_auc_score(y__test, val_prob):10.4f} | {AUCm:10.4f}")

# ROC
fpr, tpr, _ = roc_curve(y_train, train_prob)
AUC_ = auc(fpr, tpr)
fprt, tprt, _ = roc_curve(y_test, test_prob)
AUCt = auc(fprt, tprt)
fprv, tprv, _ = roc_curve(y__test, val_prob)
AUCv = auc(fprv, tprv)
plt.figure()
plt.plot(fpr, tpr, label='Train = %0.4f' % AUC_)
plt.plot(fprt, tprt, label='Test = %0.4f' % AUCt)
plt.plot(fprv, tprv, label='Validate = %0.4f' % AUCv)
plt.plot(fpr_m, tpr_m, label='McMahon = %0.4f' % AUCm)
plt.plot([0, 1], [0, 1], 'k--') # 0.5 line
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('1 - Specificity')
plt.ylabel('Sensitivity')
plt.title('ROC')
plt.legend(loc="lower right")
plt.show()

# SHAP summary plot
explainer = shap.TreeExplainer(rfc)
shap_values = explainer.shap_values(X)
shap.summary_plot(shap_values[1], X_full, color='white', max_display=X_full.shape[1])

**SHAP**

In [ ]:
# SHAP summary plot
explainer = shap.TreeExplainer(rfc)
shap_values = explainer.shap_values(X)
shap.summary_plot(shap_values[1], X, color='white', max_display=X.shape[1])

# SHAP partial dependence
fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(14,10))
shap.dependence_plot('creatinine', shap_values[1], X, interaction_index=None, ax=axes[0,0], show=False)
shap.dependence_plot('phosphate', shap_values[1], X, interaction_index=None, ax=axes[0,1], show=False)
shap.dependence_plot('platelet', shap_values[1], X, interaction_index=None, ax=axes[1,0], show=False)
shap.dependence_plot('urea', shap_values[1], X, interaction_index=None, ax=axes[1,1], show=False)
fig.subplots_adjust(wspace=0.25)
plt.show()

# SHAP force plots
arr=[]
arr=[0 for i in range(16)]
ddf = pd.DataFrame([[0]*16]*len(X), index=list(range(0,len(X))), columns=X.columns)

for i in list(range(211,219)):
  print(f"for ID = {i}:")
  # round values for increased palatability
  arr[0]=int(X.iloc[i,0]) # age
  arr[1]=int(np.round(X.iloc[i,1],0)) # AG
  arr[2]=int(np.round(X.iloc[i,2],0)) # hco3
  arr[3]=np.round(X.iloc[i,3],2) # ur
  arr[4]=np.round(X.iloc[i,4],2) # ca
  arr[5]=int(np.round(X.iloc[i,5],0)) # na
  arr[6]=int(np.round(X.iloc[i,6],0)) # cl
  arr[7]=int(np.round(X.iloc[i,7],0)) # cr
  arr[8]=np.round(X.iloc[i,8],2) # k
  arr[9]=np.round(X.iloc[i,9],2) # po4
  arr[10]=int(np.round(X.iloc[i,10],0)) # ck
  arr[11]=np.round(X.iloc[i,11],1) # hct
  arr[12]=int(X.iloc[i,12]) # sex F
  arr[13]=int(X.iloc[i,13]) # ehtn white
  arr[14]=np.round(X.iloc[i,14],1) # wbc
  arr[15]=int(np.round(X.iloc[i,15],0)) # plt
  ddf.iloc[i] = arr
  shap.force_plot(explainer.expected_value, shap_values[1][i], ddf.iloc[i], matplotlib = True)